In [1]:
import os
import numpy as np
from tqdm import tqdm
from keras.layers import Input
from keras.models import Model,Sequential
from keras.layers import Reshape,Dense,Dropout,Flatten,Conv2D,UpSampling2D
from keras.layers import LeakyReLU,ReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
import matplotlib.pyplot as plt

#(depth, input_depth, rows, cols)
K.set_image_dim_ordering('th')



latent_dim = 100
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train =  (X_train.astype(np.float32)-127.5)/127.5
X_train = X_train[:,np.newaxis,:,:]

adam = Adam(lr=0.0002,beta_1=0.5)


#Making our generator
generator = Sequential()

generator.add(Dense(128*7*7,input_dim=latent_dim))
generator.add(ReLU())
generator.add(Reshape((128,7,7)))
generator.add(UpSampling2D(size=(2,2)))
generator.add(Conv2D(64,kernel_size=(5,5),padding='same'))
generator.add(ReLU())
generator.add(UpSampling2D(size=(2,2)))
#PRODUCING A 28X28 1-CHANNEL FEATURE MAP
generator.add(Conv2D(1,kernel_size=(5,5),padding='same',activation='tanh'))
print(generator.summary())

generator.compile(loss='binary_crossentropy',optimizer=adam)



#Making our discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64,kernel_size=(5,5),padding='same',input_shape=(1,28,28),kernel_initializer=initializers.RandomNormal(stddev=0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128,kernel_size=(5,5),strides=(2,2),padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
print(discriminator.summary())
discriminator.compile(loss='binary_crossentropy',optimizer=adam)





Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 6272)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 7, 7)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 14, 14)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 14, 14)        204864    
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64, 14, 14)        0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 64, 28, 28)        0         
__________

In [2]:
discriminator.trainable = False
ganInput = Input(shape=(latent_dim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput,output=ganOutput)
gan.compile(loss='binary_crossentropy',optimizer=adam)


#Discriminator & Generator losses
dLosses = []
gLosses = []


def plotGeneratedImages(epoch,examples=100,dim=(10,10),figSize=(10,10)):
    noise = np.random.normal(0,1,size=[examples,latent_dim])
    generatedImages = generator.predict(noise)
    
    plt.figure(figSize=figSize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        plt.imshow(generatedImages[i,0])
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/dcgan_generated_images_epoch_%d.png'% epoch)
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  """


In [3]:
# Training our GAN
epochs = 2
batchSize = 128
batchCount = X_train.shape[0]/batchSize
for e in range(1,epochs+1):
    print('-'*15,'Epoch %d'%e,'-'*15)
    for i in tqdm(range(int(batchCount))):
        #creating some random noise
        noise = np.random.normal(0,1,size=[batchSize,latent_dim])
        imageBatch = X_train[np.random.randint(0,X_train.shape[0],size=batchSize)]
        #generating fake images
        generatedImages = generator.predict(noise)
        X = np.concatenate([imageBatch,generatedImages])
        #Labels for images
        yDis = np.zeros(2*batchSize)
        yDis[:batchSize] = 0.9
        
        #training the discriminator
        discriminator.trainable = True
        dloss = discriminator.train_on_batch(X,yDis)
        #Train generator
        noise = np.random.normal(0,1,size=[batchSize,latent_dim])
        yGen = np.ones(batchSize)
        discriminator.trainable = False
        gloss = gan.train_on_batch(noise,yGen)
        
    dLosses.append(dloss)
    gLosses.append(gloss)
    
    if e==1 or e%5==0:
        plotGeneratedImages(e)
#         saveModels(e)
        
        
        
        
        
        
        
        


--------------- Epoch 1 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [57:36<00:00,  7.80s/it]


TypeError: __init__() got an unexpected keyword argument 'figSize'